In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder

#import matplotlib for visualization
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
df_test=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission=pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.describe()

In [ ]:
df['Date']=pd.to_datetime(df['Date'], format='%Y-%m-%d')
df_test['Date']=pd.to_datetime(df_test['Date'], format='%Y-%m-%d')

In [ ]:
df.info()

In [ ]:
df['Country_Region'].value_counts()

In [ ]:
ax=plt.scatter(df['ConfirmedCases'],df['Fatalities'],marker=r'$\clubsuit$')
plt.xlabel('Confired Cases')
plt.ylabel('Fatalities')

In [ ]:
USCases=df.loc[df.Country_Region == 'US']

In [ ]:
NYCases=USCases.loc[USCases.Province_State=='New York']

In [ ]:
NoZeroNYCases=NYCases.loc[NYCases.ConfirmedCases > 0]

In [ ]:
fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(NYCases['Date'], NYCases['ConfirmedCases'], label='Confirmed Cases')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

# Using plot(..., dashes=...) to set the dashing when creating a line
line2, = ax.plot(NYCases['Date'], NYCases['Fatalities'], dashes=[6, 2], label='Fatalities')

ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(NoZeroNYCases['Date'], NoZeroNYCases['ConfirmedCases'], label='Confirmed Cases')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

# Using plot(..., dashes=...) to set the dashing when creating a line
line2, = ax.plot(NoZeroNYCases['Date'], NoZeroNYCases['Fatalities'], dashes=[6, 2], label='Fatalities')
ax.set_xlabel('Date')
ax.legend()
plt.show()

In [ ]:
China=df.loc[df.Country_Region=='China']

In [ ]:
Hubei=China.loc[China.Province_State=='Hubei']

In [ ]:
Hubei

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(NYCases['Date'], NYCases['ConfirmedCases'], label='New York Cases')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

# Using plot(..., dashes=...) to set the dashing when creating a line
line2, = ax.plot(Hubei['Date'], Hubei['ConfirmedCases'], dashes=[6, 2], label='Hubei Cases')

# Using set_dashes() to modify dashing of an existing line
line3, = ax.plot(NYCases['Date'], NYCases['Fatalities'], label='New York Fatalities')
line1.set_dashes([2, 2, 10, 2])  # 2pt line, 2pt break, 10pt line, 2pt break

# Using plot(..., dashes=...) to set the dashing when creating a line
line4, = ax.plot(Hubei['Date'], Hubei['Fatalities'], linestyle='--', label='Hubei Fatalities')

ax.set_ylim([1, 1000000])
plt.yscale('log')
ax.set_xlabel('Date')
ax.legend()
plt.show()

In [ ]:
def create_features(df):
    df['day'] = df['Date'].dt.day
    df['month'] = df['Date'].dt.month
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['quarter'] = df['Date'].dt.quarter
    df['weekofyear'] = df['Date'].dt.weekofyear
    return df

In [ ]:
def categoricalToInteger(df):
    #convert NaN Province State values to a string
    df.Province_State.fillna('NaN', inplace=True)
    #Define Ordinal Encoder Model
    oe = OrdinalEncoder()
    df[['Province_State','Country_Region']] = oe.fit_transform(df.loc[:,['Province_State','Country_Region']])
    return df

In [ ]:
df=categoricalToInteger(df)
df=create_features(df)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
x=['Id','Province_State','Country_Region','day','month','dayofweek','dayofyear','quarter','weekofyear']

In [ ]:
train_x=df[x]

In [ ]:
train_x.info()

In [ ]:
y=['ConfirmedCases','Fatalities']

In [ ]:
train_y=df[y]

In [ ]:
train_y

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
model_lr = linear_model.LinearRegression() 
model_lr.fit(train_x,train_y)


In [ ]:
df_test

In [ ]:
df_test=categoricalToInteger(df_test)
df_test=create_features(df_test)

In [ ]:
df_test.head()

In [ ]:
d=df_test.rename(columns={"ForecastId":"Id"})

In [ ]:
d

In [ ]:
test_x=d[x]

In [ ]:
test_x

In [ ]:
y_pred= model_lr.predict(test_x)

In [ ]:
y_pred

In [ ]:
y_pr=pd.DataFrame(y_pred, columns=['ConfirmedCases','Fatalities'])

In [ ]:
y_pr=y_pr.astype(int)

In [ ]:
y_pr

In [ ]:
df_test.head()

In [ ]:
df_test

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

# Using set_dashes() to modify dashing of an existing line
line1, = ax.plot(df_test['Date'], y_pr['ConfirmedCases'], label='Cases')

# Using plot(..., dashes=...) to set the dashing when creating a line
line2, = ax.plot(df_test['Date'], y_pr['Fatalities'],label='Fatalities')

#ax.set_ylim([1, 1000000])
#plt.yscale('log')
ax.set_xlabel('Date')
ax.legend()
plt.show()

In [ ]:
submission[['ConfirmedCases','Fatalities']]=y_pr[['ConfirmedCases','Fatalities']]

In [ ]:
submission.to_csv('submission.csv', index= False)
